In [1]:
# !git clone https://github.com/hiyouga/LLaMA-Factory.git
# %cd LLaMA-Factory
# !pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
# !pip install pickleshare
# !pip install --no-deps xformers==0.0.25
# !pip install .[bitsandbytes]

In [2]:
import torch
try:
  assert torch.cuda.is_available() is True
except AssertionError:
  print("Please set up a GPU before using LLaMA Factory: https://medium.com/mlearning-ai/training-yolov4-on-google-colab-316f8fff99c6")

# Train

In [4]:
%cd /lib/LLaMA-Factory
!llamafactory-cli train llama3_lora_orpo.yaml

/workspace/llama_factory_new
05/08/2024 09:00:02 - WARNING - llmtuner.hparams.parser - We recommend enable `upcast_layernorm` in quantized training.
05/08/2024 09:00:02 - INFO - llmtuner.hparams.parser - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, compute dtype: torch.float16
/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
[INFO|tokenization_utils_base.py:2087] 2024-05-08 09:00:02,772 >> loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--unsloth--llama-3-8b-Instruct-bnb-4bit/snapshots/89e4fd4e68bf61861110149fa59990e3bbcab6eb/tokenizer.json
[INFO|tokenization_utils_base.py:2087] 2024-05-08 09:00:02,772 >> loading file added_tokens.json from cache at None
[INFO|tokenization_utils_base.py:2087] 2024-05

# Merge

In [ ]:
import json

args = dict(
  model_name_or_path="meta-llama/Meta-Llama-3-8B-Instruct", # use official non-quantized Llama-3-8B-Instruct model
  adapter_name_or_path="saves/llama3-8b/lora/sft/",            # load the saved LoRA adapters
  template="llama3",                     # same to the one in training
  finetuning_type="lora",                  # same to the one in training
  export_dir="llama3_lora_merged",              # the path to save the merged model
  export_size=2,                       # the file shard size (in GB) of the merged model
  export_device="cuda",                    # the device used in export, can be chosen from `cpu` and `cuda`
  export_hub_model_id="Anawil/llama-3-8b-Instruct-bnb-4bit-legal-act",         # the Hugging Face hub ID to upload model
)

json.dump(args, open("merge_llama3.json", "w", encoding="utf-8"), indent=2)

%cd /workspace/llama_factory_new

!llamafactory-cli export merge_llama3.json

/workspace/llama_factory_new
/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
tokenizer_config.json: 100%|████████████████| 51.0k/51.0k [00:00<00:00, 262kB/s]
tokenizer.json: 100%|██████████████████████| 9.09M/9.09M [00:01<00:00, 7.19MB/s]
special_tokens_map.json: 100%|████████████████| 73.0/73.0 [00:00<00:00, 246kB/s]
[INFO|tokenization_utils_base.py:2087] 2024-05-08 17:14:50,546 >> loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--meta-llama--Meta-Llama-3-8B-Instruct/snapshots/e5e23bbe8e749ef0efcf16cad411a7d23bd23298/tokenizer.json
[INFO|tokenization_utils_base.py:2087] 2024-05-08 17:14:50,546 >> loading file added_tokens.json from cache at None
[INFO|tokenization_utils_base.py:2087] 2024-05-08 17:14:50,547 >> loading file s